# OpenStreetMap Data Case Study

## Map Area
Toulouse, France
- https://www.openstreetmap.org/relation/35738

As likely most students, I've chosen the city I'm living in. As a native citizen, it will be easier to detect anomalies during the audit phase and it is rather fun to explore a well-known area.


## Main Problems Encountered in the Map
To audit the OSM file, I've built several ython procedures (refer to `audit.py`) which I've applied intially to a reduced aera of Toulouse city (`map_Roseraie_Toulouse_FR`), which is actually my neighborhood.

During the preliminary audit, I've mainly focused on address tags, e.g `<TAG k='addr:*` where `* in set(['housenumber','street','city','postcode','country'])`. Main issues discovered and corrected are following:
- Inconsistent house numbering *(“2 bis, 2 Bis, 2B”)*
- Ranged or house numbering *(“43-45”)*
- Mispelled or incorrect street qualifiers *(“BD, rue, avenue”)*
- Incorrect or incomplete postal codes, Toulouse aera postal codes follows this regular expression: `^31[0-9]{3}$`
- Enumerated postal codes or house numbering *(“1;3;5” or “31200,31300”)*
- Mispelled city name

The outcome of this analysis is provided below. During this phase, i've built iteratively a renaming function based on several regular expression and string mapping which will be used during the wrangling phase (refer to `def rename_addr_subfield_value(k,v)`)

In [2]:
import audit
import os
import pandas as pd
import sqlite3

# Reduced sample map to 'Roseraie' neighborhood
# OSM_FILE = './data/map_Roseraie_Toulouse_FR'
# Full map
OSMFILE = './data/map_Toulouse_FR'

audit_result = audit.audit_addr(OSMFILE)

### Housenumber

- French house numbering often use 'bis' 'ter 'quarter' to identify different houses with the same number and located side to side ('bis' means second, 'ter' third etc.). As the spelling is not really consistent among `<tag>` values *(“B,Bis,Bis, Bis, Bis”)*, I decided to rename all these kind of numbering using only a single capital letter without space after the numbering 
- This rule is also applicable when the building identifier follows the house number
- Finnaly i've also decided to expand house numbering which are corresponding to ranges *(“43-45”)*
- Once renamed, if the house number doesn't match `RE_ADDR['housenumber'] = re.compile(r'([0-9]+)([A-Z]*)$')`, an empty list will be returned by the renaming function (see below)

In [3]:
audit_result[(audit_result.type=='housenumber') & (audit_result.new!='[]')].sort_values(by='new').head(20)

,type,value,new
30,housenumber,108 bis,['108B']
6,housenumber,108 quarter,['108Q']
94,housenumber,108 ter,['108T']
341,housenumber,10 bis,['10B']
324,housenumber,10 Bis,['10B']
242,housenumber,10 bis,['10B']
222,housenumber,10 ter,['10T']
78,housenumber,116 bis,['116B']
35,housenumber,11 bis,['11B']
215,housenumber,11 bis,['11B']


In [5]:
audit_result[(audit_result.type=='housenumber') & (audit_result.new=='[]')]

,type,value,new
274,housenumber,28B/1,[]
279,housenumber,A,[]
280,housenumber,B,[]
281,housenumber,C,[]
282,housenumber,D,[]
330,housenumber,5 Allée Antonio Machado,[]
350,housenumber,M28,[]


### Street 
- Audit and renaming for `addr:street` focuses on the street qualifier (Rue, Boulevard, etc.). I've built iteratively this list of tokens which corresponds to a correct 'road' qualifier. Any road qualifier in this list is considered as correct.

```python
EXPECTED_ROAD_TOKENS ={'Rue','Route','Autoroute','Avenue','Impasse','Chemin',
                       'Place','Rond-Point','Boulevard','Allée','Allées',
                       'Quai','Promenade','Pont','Passage','Cheminement',
                       'Esplanade','Voie','Square','Port','Passerelle','Clos',
                       'Mail','Rampe','Ponts','Cours','Côte','Descente','Caminot',
                       'Petite','Petit','Grande','Vieux','Périphérique'}
```
- As we can see, French people are very imaginative to designate road. Even as a french native, I was surprised by many of them and had to manually cross-check on google maps if it was really used.
- To rename incorrect road qualifiers:
    - Few strings had to be mapped manually 
    ```python 
        STRING_MAP_ADDR['street'] = [['ALLEE','Allée'],['Bd','Boulevard']]
    ```
    - For the vast majority, main issues were inconsistent upper/lower cases. So I applied a systematic routine to rename the first token (upper first letter and lower others ones of first token) 
    ```python 
       first_token=v.split()[0] 
       v=first_token[0].upper()+first_token[1:].lower()+v[len(first_token):]
        ``` 
    - After renaming, if the street qualifier still doesn't match, an empty list will be returned
    - Some of these street name could be processed 'angle addr_1 et addr_2'

In [6]:
audit_result[(audit_result.type=='street')].head(20)

,type,value,new
2,street,place de la bourse,['Place de la bourse']
97,street,rue des Filatiers,['Rue des Filatiers']
98,street,route de Fronton,['Route de Fronton']
99,street,rue Bida,['Rue Bida']
105,street,Angle 14 rue d'Austerlitz et 9 Boulevard de St...,[]
106,street,Angle 18 place Wilson et 2 rue d'Austerlitz,[]
107,street,Angle 6 place Wilson et rue Saint-Antoine du T,[]
108,street,Angle parking et rue du Rempart Villeneuve,[]
109,street,Angle 33 rue Lafayette et 1 rue du Rempart Vil...,[]
110,street,Angle 21 rue Croix Baragnon et rue Tolosane,[]


### postcode
- Few errors found in post codes
- Only one post code has to be mapped manually 
``` python
STRING_MAP_ADDR['postcode'] = [['3140','31400']]
```
- For others ones, 5 first digits are kept 
- Enumerates are split into a list

In [7]:
audit_result[audit_result.type=='postcode']

,type,value,new
0,postcode,31000;31100;31200;31300;31400;31500,"['31000', '31100', '31200', '31300', '31400', ..."
162,postcode,31000;31100;31200;31300;31400;31500,"['31000', '31100', '31200', '31300', '31400', ..."
171,postcode,31200‎,['31200']
172,postcode,31200‎,['31200']
321,postcode,31000;31100;31200;31300;31400;31500,"['31000', '31100', '31200', '31300', '31400', ..."
345,postcode,31700 BLAGNAC,['31700']
362,postcode,3140,['31400']
377,postcode,31000;31100;31200;31300;31400;31500,"['31000', '31100', '31200', '31300', '31400', ..."
382,postcode,31000;31100;31200;31300;31400;31500,"['31000', '31100', '31200', '31300', '31400', ..."
384,postcode,31028 Toulouse,['31028']


### city
For city, the issue was only a problem of upper/lower letter. I chose to upper first letter and lower all remaining ones.

In [8]:
audit_result[audit_result.type=='city'].head(2)

,type,value,new
95,city,toulouse,['Toulouse']
168,city,TOULOUSE,['Toulouse']


### Other Problems Encountered in the Map
- `<tag k='name'>` is widely used to identify streets name for `<way>` in lieu of `<tag k='addr:street'>` (see below)
- I tried to audit `way\tag\@name` (refer to `audit_way_tag_k_name`) and only found two misnamed streets, so I abandonned the idea to rename these two streets automaticaly as the effort was not worth it(can be done manually).
- A more interesting automated processing would be to correct `<tag>` key when it is really designating a street address (e.g replace `k='name'` per `k='addr:street'`>

- Another striking observation is that there is a huge number of tags which have very few occurrences that could be removed from the database.

In [9]:
audit_result = audit.audit_tag_k_v(OSM_FILE,parent_re='way',k_re='name|addr:')
audit_result.head(5)

,n
name,15271
addr:housenumber,1079
addr:street,1048
addr:postcode,915
addr:city,886


# Cleaning and SQL database creation
- Cleaning and creation of CSV files is achieved  by `data.py` which is adapted from lesson 013 code. I just added the procedures related to ranming of `<tag k='addr:*\' v=` values (as defined during audit). This code builds all CSV files necessary to create SQL database.
- Import to SQL database (refer to `create_sql_db.py`) is also automated:
    - creation of SQL tables is based on `SQL_SCHEMA = './data/data_wrangling_schema.sql'` which is read an excuted step by step using `sqlite3` package
    - import of CSV files into SQL database is achieved thanks to panda 'to_sql' method

    ```python
    df = pd.read_csv(csv_file)
    print('Importing ',csv_file,' to ',SQLFILE)
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html
    df.to_sql(table, conn, schema=table,if_exists='append',index=False)   
    ```
   

# Data Overview and Additional Ideas
This section contains basic statistics about the dataset, SQL queries used to gather them, and some additional ideas about the data in context.

### File sizes

In [16]:
for f in os.listdir('./data'):
    s=os.stat('./data/'+f).st_size/1024/1024
    if s>10:
        print("%s %3.1f MB" % (f,s))

for f in os.listdir('./map_Toulouse_FR/'):
    s=os.stat('./map_Toulouse_FR/'+f).st_size/1024/1024
    if s>1:
        print("%s %3.1f MB" % (f,s))
        

map_Toulouse_FR 308.8 MB
map_Toulouse_FR.db 169.9 MB
nodes.csv 95.5 MB
nodes_tags.csv 13.1 MB
ways.csv 11.7 MB
ways_nodes.csv 37.6 MB
ways_tags.csv 30.4 MB


### Number of nodes

In [17]:
SQLFILE = "./map_Toulouse_FR/map_Toulouse_FR.db"
conn = sqlite3.connect(SQLFILE)

for table in ['nodes','nodes_tags','ways','ways_tags','ways_nodes']:
    QUERY='SELECT COUNT(id) FROM '+table+';'
    df=pd.read_sql_query(QUERY, conn)
    print(table+'='+str(df.values[0][0]))

nodes=1276437
nodes_tags=328737
ways=212057
ways_tags=549209
ways_nodes=1732752


In [18]:
for table in ['nodes_tags','ways_tags']:
    QUERY='SELECT COUNT(DISTINCT(id)) FROM '+table+';'
    df=pd.read_sql_query(QUERY, conn)
    print('distinct '+table+'='+str(df.values[0][0]))

distinct nodes_tags=117356
distinct ways_tags=211142


### TOP 10 contributors

In [19]:
QUERY='''
SELECT users.user, COUNT(*) as sum
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) users
GROUP BY users.user
ORDER BY sum DESC
LIMIT 10;
'''
df=pd.read_sql_query(QUERY, conn)
df

,user,sum
0,Pinpin,490206
1,emarsden,213095
2,don-vip,114075
3,Sebastien Dinot,99562
4,Hervé TUC,75257
5,PierenBot,61672
6,square,46249
7,FredB,36225
8,isnogoud,31218
9,Florence Birée,29832


### Distinct contributors

In [20]:
QUERY='''
SELECT COUNT(uids.uid)          
FROM (SELECT uid FROM nodes UNION SELECT uid FROM ways) uids;
'''
df=pd.read_sql_query(QUERY, conn)
df.iloc[0][0]

1269

 ### Number of users appearing only once (having 1 post)

In [21]:
QUERY='''
SELECT COUNT(*) 
FROM
    (SELECT users.user, COUNT(*) as sum
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) users
     GROUP BY users.user
     HAVING sum=1)  u;
'''
df=pd.read_sql_query(QUERY, conn)
df.iloc[0][0]

293

## Complementary audit on street adresses

In [22]:
QUERY='''
SELECT DISTINCT value,count(*)
FROM (select value,key,type FROM nodes_tags  UNION SELECT value,key,type FROM ways_tags) 
WHERE type='addr' AND key='street'
;'''
df=pd.read_sql_query(QUERY, conn)
df.iloc[0][1]

1313

There are 1313 distinct `<tag k='addr:street'>` among all `<node>` and `<way>`. 

According to wikipedia https://fr.wikipedia.org/wiki/Liste_des_voies_de_Toulouse or more specific site such as https://www.annuaire-mairie.fr/rue-toulouse.html, there are more than 3000 streets in Toulouse. We have already identified that other tags can be used to define street names (such as  `<tag k='name'>` for `<way>`) but the database is very likely incomplete.
    
Below queries check that any `<node>` street adress (`<tag k='addr:street'>`) can be found in a `<way>` and conversely. 

In [23]:
QUERY='''
SELECT count(DISTINCT(value)) FROM nodes_tags
WHERE type='addr' AND key='street' AND value NOT IN (
          SELECT value
          FROM ways_tags
          WHERE (type='addr' AND key='street') OR key='name'
          )
;'''
df=pd.read_sql_query(QUERY, conn)
print('<node> address not found in <way>: '+str(df.iloc[0][0]))

QUERY='''
SELECT count(DISTINCT(value)) FROM ways_tags
WHERE type='addr' AND key='street' AND value NOT IN (
          SELECT value
          FROM nodes_tags
          WHERE (type='addr' AND key='street')
          )
;'''
df=pd.read_sql_query(QUERY, conn)
print('<way> address not found in <node>: '+str(df.iloc[0][0]))

<node> address not found in <way>: 202
<way> address not found in <node>: 173


However if we have a look to first 10 street names in the first category, we likely face inconsistencies due to spelling human errors. It might not be obvious for an alien to Toulouse city but a lot of these street names are misspelled. For instance:
- *'Grande Rue Saint'* should be ***Grande Rue Saint-Michel***
- *'Rue d'Alsace'* should be ***Rue d'Alsace-Lorainne***

In [24]:
QUERY='''
SELECT DISTINCT(value) FROM nodes_tags
WHERE type='addr' AND key='street' AND value NOT IN (
          SELECT value
          FROM ways_tags
          WHERE (type='addr' AND key='street') OR key='name'
          )
;'''
df=pd.read_sql_query(QUERY, conn)

df.head(10)

,value
0,Route de Fronton
1,Grande Rue Saint
2,Rue d'Alsace
3,Place Intérieure Saint
4,Allées Paul Sabatier
5,Rue de Remusat
6,Place de l'Église Saint
7,Avenue Etienne Billières
8,Grande rue Saint
9,Rue de la Chaine


### By the way, am i able to find my street?

In [25]:
QUERY='''
SELECT DISTINCT id,key,type,value
FROM (select id,value,key,type FROM nodes_tags  UNION SELECT id,value,key,type FROM ways_tags) 
WHERE (type='addr' AND key='street' AND value='Rue des Pavots') OR (key='name' AND value='Rue des Pavots')
;'''
df=pd.read_sql_query(QUERY, conn)

df.head(10)

,id,key,type,value
0,23234316,name,regular,Rue des Pavots


Bingo and it seems to be defined as a `<way><tag>`. Can we find all tags in my `<way>`? 

In [26]:
QUERY='''
SELECT id,key,value,type
FROM ways_tags 
WHERE id = (SELECT DISTINCT id FROM ways_tags WHERE (key='name' AND value='Rue des Pavots'))
;'''
df=pd.read_sql_query(QUERY, conn)

df

,id,key,value,type
0,23234316,highway,residential,regular
1,23234316,name,Rue des Pavots,regular
2,23234316,surface,asphalt,regular


Not very fancy, let's check all `<node><tag>` defined in my `<way>`

In [27]:
QUERY='''
SELECT ways_nodes.id,ways_nodes.node_id,ways_nodes.position,nodes_tags.key,nodes_tags.value
FROM ways_nodes JOIN nodes_tags ON ways_nodes.node_id=nodes_tags.id
WHERE ways_nodes.id = (SELECT id FROM ways_tags WHERE (key='name' AND value='Rue des Pavots'))
;'''
df=pd.read_sql_query(QUERY, conn)

df

,id,node_id,position,key,value
0,23234316,251370084,3,created_by,JOSM
1,23234316,251370043,11,created_by,JOSM


Still not very fancy and surprisingly enough no house number can be found here. I cross-checked on https://www.openstreetmap.org/node/208025613 and we can see that houses are defined as children elements of a `<relation>` for 'Rue des Pavots' street. This might indicate that we have many orphans `<node>` in our database (as we didn't neither analysed nor used `<relation>` in our database).
    
Let's check that. First for this particular `<relation>` (my street), is `<node id='2080256136'>` (not my real house by the way :)) in the SQL database?

In [28]:
QUERY='''
SELECT id,lat,lon,user,uid,version,changeset,timestamp
FROM nodes
WHERE id=2080256136
;'''
df=pd.read_sql_query(QUERY, conn)
df

,id,lat,lon,user,uid,version,changeset,timestamp
0,2080256136,43.625372,1.473654,isnogoud,78608,1,14381631,2012-12-23T18:35:53Z


I can find this node (hopefully) and as expected this node is an orphan (not a child `<node>` of any `<way>`):

In [29]:
QUERY='''SELECT node_id,id FROM ways_nodes WHERE node_id=2080256136;'''
df=pd.read_sql_query(QUERY, conn)
df

,node_id,id


What's the total number of these orphan nodes?

In [30]:
QUERY='''SELECT COUNT(*) FROM (SELECT id FROM nodes EXCEPT SELECT node_id FROM ways_nodes);'''
df=pd.read_sql_query(QUERY, conn)
df

,COUNT(*)
0,69721


Are there any valuable information (`<tag>`) in these orphan `<node>`?

In [31]:
QUERY='''
SELECT id,key,type,value
FROM nodes_tags
WHERE id=(SELECT id FROM nodes EXCEPT SELECT node_id FROM ways_nodes)
;'''
df=pd.read_sql_query(QUERY, conn)
df

,id,key,type,value
0,8831758,city,addr,Toulouse
1,8831758,postcode,addr,31500
2,8831758,street,addr,Rue des Cheminots
3,8831758,amenity,regular,theatre
4,8831758,phone,contact,+33561483829
5,8831758,website,contact,http://theatrelehangar.over-blog.fr/categorie-...
6,8831758,name,regular,Théâtre Le Hangar
7,8831758,source,regular,GrandToulouse
8,8831758,date,source,2011-11-06


Very few. Actually there is only one single `<node id='8831758'>` with some tagged information. If we don't process `<relation>`, all these orphans nodes are useless and can be removed from the database.

## Additional Data Exploration

### Top 10 amenities or leisure or shop

In [32]:
QUERY='''
SELECT key, value, COUNT(*) as num
FROM nodes_tags
WHERE key='leisure' or key='amenity' or key='shop'
GROUP BY value
ORDER BY num DESC LIMIT 10
;'''
df=pd.read_sql_query(QUERY, conn)
df

,key,value,num
0,amenity,recycling,1074
1,amenity,bench,832
2,amenity,restaurant,712
3,amenity,waste_basket,490
4,shop,clothes,425
5,amenity,bicycle_parking,420
6,amenity,fast_food,325
7,amenity,post_box,318
8,amenity,bicycle_rental,285
9,shop,hairdresser,222


### Identify all leisure, amenity and shops within 1km from my house

This is the most challenging set of SQL queries I've ever created :)

I've followed following steps:
- Creation of a SQL VIEW gathering all nodes_tags and ways_tags 
- Creation of a SQL VIEW gathering all amenities, shops and leisure (`ameshosure`) with full adresses is created. All elements with an incomplete adress won't be part of this table. As this filter is rather drastic (due to incompleteness of Open street map), the address is limited to street name and city (no zip code and no house numb

In [33]:
try:
    conn.execute('DROP VIEW all_tags')
except Exception as e: print(e)
try:
    conn.execute('DROP VIEW ameshosure')
except Exception as e: print(e)


QUERY='''
CREATE VIEW all_tags AS
 SELECT *
 FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags)
;'''

try:
    conn.execute(QUERY)
except Exception as e: print(e)

QUERY='''
CREATE VIEW ameshosure AS
    SELECT all_tags.id as node_or_way_id, 
           all_tags.key,
           all_tags.value,
           name.value as name,
           street.value as street,
           city.value as city
           
    FROM all_tags
        JOIN (SELECT DISTINCT(id),value FROM all_tags WHERE value!='private') as pv
        JOIN (SELECT DISTINCT(id),value FROM all_tags WHERE key='name') as name
        JOIN (SELECT DISTINCT(id),value FROM all_tags WHERE key='street') as street
        JOIN (SELECT DISTINCT(id),value FROM all_tags WHERE key='city') as city
        
    
    ON  all_tags.id=pv.id 
        AND all_tags.id = name.id 
        AND all_tags.id = street.id
        AND all_tags.id = city.id
        
    WHERE (all_tags.key='leisure' or  all_tags.key='amenity' or  all_tags.key='shop')
    
    GROUP BY all_tags.id
;'''
try:
    conn.execute(QUERY)
except Exception as e: print(e)


In [34]:
QUERY='''
SELECT * FROM ameshosure LIMIT 5
;'''
df=pd.read_sql_query(QUERY, conn)
df

,node_or_way_id,key,value,name,street,city
0,8831758,amenity,theatre,Théâtre Le Hangar,Rue des Cheminots,Toulouse
1,18505705,amenity,school,Groupe scolaire Marie Laurencin,Route de Mons,Balma
2,22689423,leisure,stadium,Palais des Sports de Toulouse,Rue du Canon d'Arcole,Toulouse
3,22718101,amenity,school,Lycée général Saint-Sernin,Place Saint,Toulouse
4,22718102,amenity,school,Lycée général et technologique Ozenne,Rue Merly,Toulouse


Once this SQL VIEW is created, we can launch the main query (see below). The distance between nodes is calculated using `def haversine(lon1, lat1, lon2, lat2):`. This function is exported to SQL.

In [35]:
# http://jonisalonen.com/2014/computing-distance-between-coordinates-can-be-simple-and-fast/
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

#https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.create_function
conn.create_function("haversine", 4, haversine)

QUERY='''
SELECT key,value,name,street,city
FROM
(
     SELECT nodes.id  
     FROM nodes,nodes_tags
     ON nodes_tags.id=nodes.id 
     WHERE haversine(lon,lat,(SELECT lon FROM NODES WHERE ID=2080256136),(SELECT lat FROM NODES WHERE ID=2080256136))<1 
     AND (nodes_tags.key='leisure' or  nodes_tags.key='amenity' or  nodes_tags.key='shop')
     GROUP BY nodes.id

    UNION ALL

    SELECT ways_nodes.id
    FROM nodes JOIN ways_nodes JOIN ways_tags
    ON nodes.id=ways_nodes.node_id AND ways_tags.id=ways_nodes.id 
    WHERE haversine(lon,lat,(SELECT lon FROM NODES WHERE ID=2080256136),(SELECT lat FROM NODES WHERE ID=2080256136))<1 
    AND (ways_tags.key='leisure' or  ways_tags.key='amenity' or  ways_tags.key='shop')
    GROUP BY ways_nodes.id
) ids, ameshosure
ON ameshosure.node_or_way_id=ids.id
;'''


df=pd.read_sql_query(QUERY, conn)
df

,key,value,name,street,city
0,amenity,school,Collège Jolimont,Chemin Cassaing,Toulouse
1,shop,frozen_food,Picard,Chemin de Gabardie,Toulouse
2,shop,butcher,Direct Viandes,Route d'Agde,Toulouse
3,amenity,fast_food,Burger King,Route d'Agde,Toulouse
4,amenity,post_office,Toulouse Roseraie,Rue Louis Plana,Toulouse
5,amenity,recycling,Déchetterie d'Atlanta,Chemin de Gramont,Toulouse
6,amenity,restaurant,Café à la Une,Rue Louis Plana,Toulouse
7,amenity,library,Bibliothèque de quartier - Roseraie,Rue de Périole,Toulouse
8,amenity,library,Desserte Bibliobus Amouroux,Rue Roubichou,Toulouse
9,leisure,laser,Laser Quest,Route d'Agde,Toulouse


In [37]:
conn.close()

This list gathers all amenities, shops and leisures within 1km from '20 Rue des Pavots' which have an address defined. But unfortunately a lot of them are missing. **Where is gone my bakery?**

Actually my bakery corresponds to this node:
https://www.openstreetmap.org/node/2258458127
which is unfortunately not linked to any `<way>`, `<relation>` and doesn't have any address `<tag>` defined. So how open streep map manages to find it?
https://www.openstreetmap.org/search?query=boulangerie%20de%20la%20roseraie#map=19/43.62470/1.47516

That's the strength of open street map ... even if the database is neither complete nor strongly structured, reverse geo localization through GPS coordinates allows to retrieve detailed street addresses. This is not a straightforward SQL query! 

# Additional Ideas

## Reverse geolocalisation

As highlighted above, we were not able to identify  all amenities, shops and leisures in the vicinity of '20 Rue des Pavots' because their addresses were not fully filled in the database. To overcome this issue, Open Street Map has built reverse geolocalization functionalities on top of the database. These functions allows to retrieve the nearest address from a GPS coordinate.
https://wiki.openstreetmap.org/wiki/Nominatim

If we come back to our example we could identify all candidates in the database if we forget adresses and just focus on GPS coordinates. Once we have GPS coordinates for each node, we can retrieve the address of each amenity, shop or leisure using a reverse geolocalisation query. Such queries can be quite easily be achieved through an external API, see for example:
https://github.com/slawek87/geolocation-python/

But this approach is not fully satisfactory as our database won't ever be self-contained. A more elegant way to proceed would be to build our own reverse geolocalisation functionnality based on database content only. But this is another project.

## Database cleaning & completion

During our queries, we stumbed upon key issues of this OSM to SQL transformation:
- Many `<way>` describing street/road are using `<tag k='name'>` instead of `<tag k='addr:street'>`. 
- Many `<tag k='addr:street'>` names are still misspelled.
- `<relation>` have not been analysed and integrated in the SQL database
- Many `<tag>` are seldom used an could be removed from the database

Regarding first two items, it could be quite easily corrected using an external reference such as https://www.annuaire-mairie.fr/rue-toulouse.html. This reference identifies spelling of all streets of Toulouse.

For missing tables, we might get inspiration from the original SQL OSM format.
https://wiki.openstreetmap.org/wiki/Database

For deprecated or poorly used `<tag>`, I don't know if there is an easy solution and I fear this is part of OSM folklore. Distributon of tags is really skewed. So it's dificuly to decide which part of the tail would need to be cut. On top of that, from perfomance standpoint, as volume of these tags is really low, it shouldn't make much difference. 

Finally the configuration management of the databse has not been explored at all in this review. For a light and self-contained databse, it would be necessary to keep only the latest versions of each element of the database. Such a processing would likely produce a lighter dabase.

# Conclusion


 This deep dive in the OSM ocean has been a very interesting trip. This format is not strongly structured and there are many different ways to modelize the exact same map. These degree of freedom increase the difficulty of automated processing of queries. However, the strength of this format relies on a huge set of GPS coordinates which provides a very robust basement for any smart processing such as reverse geolocalisation. The weakness of this format is that the database embodies many human typing errors on streets or building denomination. Even if OSM is mainly built through automated processing of external databases. These source databases are themselves not exempt of defect. And, on top of this automated processing, manual edition might worsen the situation. Anyhow this open format is really powerfull and the quality and map details are very impressive.
